# Baseline solution based on GPT-4 prompting

## Imports and data read

In [116]:
import pandas as pd
import numpy as np
import pprint

from tqdm import tqdm
from openai import OpenAI
from sklearn.metrics import accuracy_score, f1_score

## Pipeline

https://platform.openai.com/docs/api-reference/authentication

In [2]:
api_key = 'sk-proj-7yyg4KC696T7WK6CmLeWT3BlbkFJfjiC6bs5r8R7yoGLCIUP'

In [3]:
client = OpenAI(api_key=api_key)

In [5]:
def analyze_sentiment(comment, system_prompt):
    """
    Sends a request to OpenAI's GPT model to analyze sentiment.
    """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": comment}
        ]
    )   
    return response.choices[0].message.content

In [11]:
df = pd.read_parquet('./data_provided/final_dataset/final_17042025.parquet')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12224 entries, 0 to 12223
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   response_id           12224 non-null  int64 
 1   document_id           12224 non-null  int64 
 2   user_id               12224 non-null  int64 
 3   annotator_sentiment   12224 non-null  object
 4   is_ck_annotation      12224 non-null  int64 
 5   response_timestamp    12224 non-null  object
 6   document_content      12224 non-null  object
 7   annotation_date       12224 non-null  object
 8   username              12224 non-null  object
 9   unique_document_id    12224 non-null  object
 10  language_wc           12224 non-null  object
 11  document_length       12224 non-null  int64 
 12  gpt_labels_v1         12224 non-null  object
 13  language_gpt          12224 non-null  object
 14  language_manual       12224 non-null  object
 15  language              12224 non-null

In [13]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels_v1,language_gpt,language_manual,language,stratification_label,df_set
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,Ukrainian,ukrainian,ua,neutral_ua,validation
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,Code-mixed,russian,ru,negative_ru,test
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,negative,Code-mixed,russian,ru,negative_ru,train
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,Ukrainian,russian,ua,neutral_ua,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,Code-mixed,ukrainian,ua,negative_ua,train
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,positive,Ukrainian,ukrainian,ua,mixed_ua,train
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,Ukrainian,ukrainian,ua,negative_ua,train


In [14]:
df.annotator_sentiment.unique()

array(['neutral', 'negative', 'positive', 'mixed'], dtype=object)

## Metrics

In [15]:
def evaluate_gpt_results(df):
    """
    Computes accuracy and F1-score for GPT sentiment analysis results.
    
    Parameters:
    df (pd.DataFrame): A DataFrame containing 'true_label' and 'gpt_label'.
    
    Returns:
    dict: A dictionary with accuracy and F1-score.
    """
    accuracy = accuracy_score(df['y'], df['y_hat'])
    f1 = f1_score(df['y'], df['y_hat'], average='weighted')
    
    return {"accuracy": round(accuracy, 4), "f1_score": round(f1, 4)}

## System prompt creation - zero shot approach

In [16]:
system_prompt = '''

You are a sentiment analysis expert. You need to analyze peoples emotions and define the comments sentiment from the Ukrainian social media. The input you receive is a text from Telegram messenger. The text can be Ukrainian or Russian. 

Your output must be a single word from the following list: ['positive', 'neutral', 'negative', 'mixed']. 

'''

In [17]:
print(len(system_prompt)/4)

86.75


In [ ]:
gpt_answers = []

for id in tqdm(df.index[:1000]):
    gpt_answers.append(analyze_sentiment(df.loc[id, 'document_content'], system_prompt=system_prompt))

In [ ]:
df['gpt_labels'] = gpt_answers

In [ ]:
df['gpt_labels'].unique()

array(['positive', 'negative', 'mixed', 'neutral', 'negative '],
      dtype=object)

In [ ]:
df.loc[df['gpt_labels'] == 'negative ', 'gpt_labels'] = 'negative'

In [ ]:
df['gpt_labels'].unique()

array(['positive', 'negative', 'mixed', 'neutral'], dtype=object)

In [ ]:
df.gpt_labels.value_counts()

gpt_labels
negative    494
positive    289
neutral     151
mixed        66
Name: count, dtype: int64

In [ ]:
df.annotator_sentiment.value_counts()

annotator_sentiment
negative    364
neutral     360
positive    212
mixed        64
Name: count, dtype: int64

In [ ]:
df['y'] = df['annotator_sentiment'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [ ]:
df['y_hat'] = df['gpt_labels'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [41]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels_v1,language_gpt,language_manual,language,stratification_label,df_set
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,Ukrainian,ukrainian,ua,neutral_ua,validation
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,Code-mixed,russian,ru,negative_ru,test
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,negative,Code-mixed,russian,ru,negative_ru,train
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,Ukrainian,russian,ua,neutral_ua,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,Code-mixed,ukrainian,ua,negative_ua,train
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,positive,Ukrainian,ukrainian,ua,mixed_ua,train
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,Ukrainian,ukrainian,ua,neutral_ua,train
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,Ukrainian,ukrainian,ua,negative_ua,train


In [ ]:
metrics = evaluate_gpt_results(df)

In [45]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.585, 'f1_score': 0.5609}


## System prompt creation - few shots approach - Experiment 1

In [9]:
system_prompt = '''

You are a sentiment analysis expert. You need to analyze peoples emotions and define the comments sentiment from the Ukrainian social media. The input you receive is a text from Telegram messenger. The text can be Ukrainian or Russian. 

Your output must be a single word from the following list: ['positive', 'neutral', 'negative', 'mixed']. 
Here are some examples of inputs to you and your outputs expected:

```
Input: 'Киеве продолжать звучать взрывы. СМИ сообщают, что в ряде районов пропал свет.'
Output: 'negative'

Input: 'Через обстріли у Києві знеструмлено майже 260 тисяч споживачів, відключалося обладнання підстанцій у Київській області, — Міненерго   У Києві пошкоджено повітряну лінію, без напруги низка підстанцій – знеструмлені 259 тисяч точок обліку.'
Output: 'negative'

Input: 'Попередній висновок🧐  Перше. У нас найкраще в світі ППО! Жодна країна не витримала б такої масованої атаки. Досвід і наявність засобів вберігають 99% населення  Друге. Русня просто вчергове хоче нас залякати, але минулої зими і в травні ми чітко вивчили алгоритми безпеки і хєр кладем русакам на воротнік  Третє. Не бійтеся, скоро і вода і електрика повернуться, а воля наша не зламається!  Інформує Київ | Надіслати новину'
Output: 'positive'

Input: 'Голосеевский р-н есть свет 😉✌🏻'
Output: 'positive'

Input: 'Підкажіть, будь ласка, чи бувають на Видубичах на платформах міської і приміських електричок?'
Output: 'neutral'

Input: 'З цього ресторану доставляють дуже швидко! Але їжа у них може бути несмачна, тому рекомендувати складно...'
Output: 'mixed'
```

Remember that the 'sentiment' values can be only and only from the following list: ['positive', 'neutral', 'negative', 'mixed']. 

'''

In [ ]:
print(len(system_prompt)/4)

In [ ]:
gpt_answers = []

for id in tqdm(df.index):
    gpt_answers.append(analyze_sentiment(df.loc[id, 'document_content']))

100%|██████████| 12224/12224 [1:52:13<00:00,  1.82it/s]  


In [ ]:
gpt_answers

['нейтральний',
 'Негативний',
 'Негативний',
 'змішаний',
 'Нейтральний',
 'Позитивний',
 'Позитивний',
 'Нейтральний',
 'Змішаний',
 'нейтральний',
 'негативний',
 'негативний',
 'змішаний',
 'Негативний',
 'негативний',
 'Нейтральний',
 'позитивний',
 'негативний',
 'Позитивний',
 'негативний',
 'Позитивний',
 'позитивний',
 'Позитивний',
 'негативний',
 'негативний',
 'негативний',
 'негативний',
 'змішаний',
 'негативний',
 'негативний',
 'Змішаний',
 'змішаний',
 'негативний',
 'Нейтральний',
 'Нейтральний',
 'Позитивний',
 'змішаний',
 'Позитивний',
 'нейтральний',
 'Позитивний',
 'Позитивний',
 'Позитивний',
 'позитивний',
 'змішаний',
 'Негативний',
 'позитивний',
 'нейтральний',
 'Негативний',
 'Позитивний',
 'негативний',
 'Позитивний',
 'Нейтральний',
 'Позитивний',
 'негативний',
 'Негативний',
 'Позитивний',
 'нейтральний',
 'Негативний',
 'Нейтральний',
 'Негативний.',
 'Змішаний',
 'нейтральний',
 'змішаний',
 'нейтральний',
 'негативний',
 'негативний',
 'Негативний',


In [ ]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11
...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114


In [ ]:
df['gpt_labels'] = gpt_answers

In [102]:
df.to_parquet('./data_provided/final_dataset/gpt_labeled_with_language.parquet')

In [234]:
df = pd.read_parquet('./data_provided/final_dataset/gpt_labeled_with_language.parquet')

In [235]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels,language_gpt,language_explanation_gpt
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,Ukrainian,n/a
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,Ukrainian,n/a
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,Code-mixed,n/a
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,negative,Code-mixed,n/a
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,Ukrainian,n/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,Code-mixed,n/a
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,positive,Ukrainian,n/a
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,Ukrainian,n/a
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,Ukrainian,n/a


In [236]:
df.gpt_labels.unique()

array(['neutral', 'negative', 'positive', 'mixed'], dtype=object)

In [81]:
df.loc[(df.gpt_labels == """Input: "Києве продовжуються вибухи та відключення світла в різних районах міста. Жителі скаржаться на невпевненість у майбутньому."\nOutput: \'negative\'"""), 'gpt_labels'] = 'negative'

In [95]:
df.loc[(df.gpt_labels == """Звісно, напишіть текст, який ви бажаєте проаналізувати на настрій!"""), 'gpt_labels'] = 'negative'

In [96]:
df.loc[(df.gpt_labels == """Моя мова зараз українська. Я завжди готова допомогти! Як я можу вам сьогодні допомогти?"""), 'gpt_labels'] = 'negative'

In [98]:
df.loc[(df.gpt_labels == """1. 'negative'\n2. 'negative'\n3. 'neutral'"""), 'gpt_labels'] = 'mixed'

In [237]:
df['y'] = df['annotator_sentiment'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [238]:
df['y_hat'] = df['gpt_labels'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [239]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels,language_gpt,language_explanation_gpt,y,y_hat
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,Ukrainian,n/a,0,0
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,Ukrainian,n/a,0,-1
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,Code-mixed,n/a,-1,-1
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,negative,Code-mixed,n/a,-1,-1
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,Ukrainian,n/a,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,Code-mixed,n/a,-1,1
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,positive,Ukrainian,n/a,2,1
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,Ukrainian,n/a,0,0
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,Ukrainian,n/a,-1,-1


In [240]:
metrics = evaluate_gpt_results(df)

In [241]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.6185, 'f1_score': 0.6044}


By language

In [251]:
df.language_wc.unique()

array(['uk', 'ru'], dtype=object)

In [252]:
metrics = evaluate_gpt_results(df.loc[df.language_wc == 'uk'])

In [253]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.6008, 'f1_score': 0.5829}


In [254]:
metrics = evaluate_gpt_results(df.loc[df.language_wc == 'ru'])

In [255]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.6578, 'f1_score': 0.6544}


By sentiment

In [242]:
df.annotator_sentiment.unique()

array(['neutral', 'negative', 'positive', 'mixed'], dtype=object)

In [243]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'neutral'])

In [244]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.43, 'f1_score': 0.6014}


In [245]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'negative'])

In [246]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.8128, 'f1_score': 0.8967}


In [247]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'positive'])

In [248]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.7417, 'f1_score': 0.8517}


In [249]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'mixed'])

In [250]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.1447, 'f1_score': 0.2529}


## System prompt creation - few shots approach - Experiment 2

In [6]:
system_prompt = '''

Ти - експерт із визначення тональності тексту. Наше завдання - максимально точно визначати емоцію (сентимент), яку людина закладає у написаний текст. Для цього я показуватиму тобі тексти з українських соціальних мереж, а ти - обиратимеш вірний варіант відповіді щодо сентименту. Варіанти відповідей будуть наступні:

1. Позитивний -> Використані вирази, що відображають позитивні емоції (радість, підтримку, захоплення тощо);
2. Негативний -> Використані вирази, що відображають негативні емоції (критика, сарказм, осуд, агресія, сумнів, страх тощо);
3. Нейтральний -> Автор не використовує ні позитивних, ні негативних виразів (нейтральна емоція тексту);
4. Змішаний -> Текст містить вирази як з позитивного спектру емоцій, так і з негативного (змішаний випадок);


Важливо, що потрібно вказувати не власну здогадку щодо сентименту автора, а знаходити вказівки на нього у конкретних виразах. Надам декілька прикладів. Приклади:
" Аварії " -> цей короткий текст має нейтральний сентимент. Попри те, що слово “аварії” часто має негативний контекст, у даному випадку відсутня будь-яка додаткова інформація, що відображає сентимент автора.
" Так я ж тебе задал вопрос. Киев, май, первое применение пэтриотов - когда все небо осветили этим - были там и х22, и кинжалы - так были прилеты тогда? Не было. Вопрос залу - почему так произошло?  Пэтриоты сбивают всю эту срань " -> цей текст має негативний сентимент. Попри те, що факт “Петріоти збивають ракети” може відчуватись позитивно, автор використовує вирази, що характеризують агресію та критику до співрозмовника.
" Зникло світло у Святошинському районі. " -> даний текст має нейтральний сентимент. Сам факт відсутності електроенергії сприймається негативно, але автор тексту не використовує ні позитивних, ні негативних слів / виразів.
" Проблеми зі світлом в Києві та області після вибухів! " -> у свою чергу наступна новина має негативне забарвлення. Автор демонструє своє відношення через слово “Проблеми” та знак оклику “!”, підкреслюючи експресію.
" :cry: Внаслідок ракетної атаки зафіксовано падіння уламків в Печерському районі на дах багатоповерхового житлового будинку, – КМВА " -> текст із негативним сентиментом, що автор демонструє через використання “:cry:” емодзі.
" Ну норм " -> це приклад позитивного сентименту. Сам текст не є сильно експресивним, але автор явно демонструє емоцію “схвалення” чогось, яка належить до позитивного спектру.
" :exclamation:В бік Києва пуски ще декількох ‘Кинджалів’. Ворог намагається пробити наші ППО. Поки відбиваємося, але є падіння уламків, тож перебуваємо в укриттях або хоча б за парою стін. " -> дана новина є прикладом негативного сентименту. Автор демонструє своє відношення до події через вирази “Ворог намагається пробити наші ППО”, “Поки відбиваємося, але…”.
" С чего ты взял? У меня в Ирпене все окна повыбивало я сохранил квитанцию то что сам поставил и вернули 20.000‍ " -> приклад “змішаного” сентименту. У першій частині автор демонструє критику по відношенню до іншої людини. У другій частині тексту - автор радіє, що йому компенсовано витрати на відновлення домівки.


Твоя відповідь має бути лише одним словом. ЦЕ ВАЖЛИВО! Ти маєш відповідати виключно лише одним словом: позитивний, негативний, нейтральний, змішаний. 

'''

In [7]:
print(len(system_prompt)/4)

810.0


In [ ]:
gpt_answers = []

for id in tqdm(df.index):
    gpt_answers.append(analyze_sentiment(df.loc[id, 'document_content']))

100%|██████████| 12224/12224 [1:52:13<00:00,  1.82it/s]  


In [ ]:
df['gpt_labels'] = gpt_answers

In [206]:
df.to_parquet('./data_provided/final_dataset/gpt_labeled_2_1.parquet')

In [207]:
df = pd.read_parquet('./data_provided/final_dataset/gpt_labeled_2_1.parquet')

In [208]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,mixed
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,neutral
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative


In [209]:
df['gpt_labels'].value_counts()

gpt_labels
negative    5150
positive    3298
neutral     2378
mixed       1398
Name: count, dtype: int64

In [210]:
df['y'] = df['annotator_sentiment'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [211]:
df['y_hat'] = df['gpt_labels'].apply(lambda x: {'negative': -1, 'neutral': 0, 'positive': 1, 'mixed': 2}[x])

In [212]:
df

,response_id,document_id,user_id,annotator_sentiment,is_ck_annotation,response_timestamp,document_content,annotation_date,username,unique_document_id,language_wc,document_length,gpt_labels,y,y_hat
0,1,1,277133851,neutral,1,2025-03-09T23:23:07.220881,⚡️Українська делегація відправилася на перемов...,2025-03-09,O,1_1,uk,67,neutral,0,0
1,3,2,1065283664,neutral,1,2025-03-09T23:44:28.262307,"Вибухи на Одещині, попередньо — ППО.",2025-03-09,A,2_1,uk,36,negative,0,-1
2,4,3,1065283664,negative,1,2025-03-09T23:45:00.503098,"А что делать тем ,кто лишился своего жилья ,по...",2025-03-09,A,3_1,ru,177,negative,-1,-1
3,5,4,1065283664,negative,1,2025-03-09T23:46:33.265766,Тогда учись быстро бегать. Для меня вопрос сло...,2025-03-09,A,4_1,ru,103,mixed,-1,2
4,6,5,1065283664,neutral,1,2025-03-09T23:46:38.993496,Добрий день,2025-03-09,A,5_1,uk,11,neutral,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12219,13028,8948,467130971,negative,0,2025-04-04T08:02:37.362562,"Краще ""повинна бути зручнішою, ніж Uber чи Boo...",2025-04-04,D,8948_0,uk,51,positive,-1,1
12220,13029,2094,467130971,mixed,0,2025-04-04T08:03:35.792932,Увага! З деяких інтернет джерел шириться інфор...,2025-04-04,D,2094_0,uk,402,neutral,2,0
12221,13030,5013,467130971,neutral,0,2025-04-04T08:03:42.008533,"Питання, цей сертифікат можна вже використовув...",2025-04-04,D,5013_0,uk,113,neutral,0,0
12222,13031,4572,467130971,negative,0,2025-04-04T08:03:48.251166,На Вугледарському напрямку загинув Рома Іванен...,2025-04-04,D,4572_0,uk,114,negative,-1,-1


In [213]:
metrics = evaluate_gpt_results(df)

In [214]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.5722, 'f1_score': 0.5748}


By language

In [217]:
df.language_wc.unique()

array(['uk', 'ru'], dtype=object)

In [219]:
metrics = evaluate_gpt_results(df.loc[df.language_wc == 'uk'])

In [220]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.5765, 'f1_score': 0.5724}


In [221]:
metrics = evaluate_gpt_results(df.loc[df.language_wc == 'ru'])

In [222]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.5625, 'f1_score': 0.5835}


By sentiment

In [223]:
df.annotator_sentiment.unique()

array(['neutral', 'negative', 'positive', 'mixed'], dtype=object)

In [226]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'neutral'])

In [227]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.386, 'f1_score': 0.557}


In [228]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'negative'])

In [229]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.7058, 'f1_score': 0.8275}


In [230]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'positive'])

In [231]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.7581, 'f1_score': 0.8624}


In [232]:
metrics = evaluate_gpt_results(df.loc[df.annotator_sentiment == 'mixed'])

In [233]:
print("Evaluation Metrics:", metrics)

Evaluation Metrics: {'accuracy': 0.2878, 'f1_score': 0.447}


# Conslusions

* English promtps work better
* Best results are achieved
* <xml> tags in prompts don't fully resolve problems with possible prompt injections 
* The best approach: few-shot examples, English prompt,  similar to guidelines for people